<a href="https://colab.research.google.com/github/rafasyafiq/pyda-online/blob/master/day2/004_Data_Cleaning_and_Preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np

In [0]:
import builtins

def print(*args, **kwargs):
    builtins.print(*args, **kwargs, end='\n\n')


## Handling Missing Data

The way that missing data is represented in pandas objects is somewhat imperfect, but it is functional for a lot of users. For numeric data, pandas uses the floating-point value **NaN** (Not a Number) to represent missing data. We call this a **sentinel value** that can be easily detected

In [0]:
string_data = pd.Series(['aardvark', 'artichoke', np.nan, 'avocado'])
string_data

In [0]:
string_data.isnull()

In [0]:
string_data.isnull().sum()

In [0]:
# The built-in Python None value is also treated as NA in object arrays

string_data[0] = None
string_data.isnull()

In [0]:
pd.options.display.max_colwidth = 500
df = pd.read_csv('https://raw.githubusercontent.com/rafasyafiq/pyda-online/master/Data/NA_handling_methods.csv', names=['Description'], sep='\n')
df.index = list(range(1, 5))

def f1(x):
    x = x.split(' ')
    return x.pop(0)


df['Argument'] = df['Description'].map(f1) # get the first word

def f2(x):
    x = x.split(' ')
    return " ".join(x[1:])

df['Description'] = df['Description'].map(f2) # remove the first word

df = df.reindex(columns=['Argument', 'Description'])
print("NA handling methods")
df

### Filtering Out Missing Data

There are a few ways to filter out missing data. While you always have the option to do it by hand using **pandas.isnull** and **boolean indexing**, the **dropna** can be helpful.

In [0]:
# On a Series, it returns the Series with only the non-null data and index values

from numpy import nan as NA

In [0]:
data = pd.Series([1, NA, 3.5, NA, 7])

In [0]:
data.dropna()
# data[data.notnull()]

With DataFrame objects, things are a bit more complex. You may want to drop rows or columns that are all NA or only those containing any NAs. dropna by **default drops any row containing a missing value**

In [0]:
data = pd.DataFrame([[1., 6.5, 3.], [1., NA, NA],
                     [NA, NA, NA], [NA, 6.5, 3.]])
data

In [0]:
cleaned = data.dropna()
cleaned

In [0]:
# Passing how='all' will only drop rows that are all NA

data.dropna(how='all')
# data.dropna(how='any')

In [0]:
# To drop columns in the same way, pass axis=1

data[4] = NA
data

In [0]:
data.dropna(axis=1, how='all')

A related way to filter out DataFrame rows tends to concern time series data. **Suppose you want to keep only rows containing a certain number of observations**. You can indicate this with the **thresh** argument

In [0]:
df = pd.DataFrame(np.random.randn(7, 3))
df.iloc[:4, 1] = NA
df.iloc[:2, 2] = NA
df

In [0]:
print(df.dropna())
print(df.dropna(thresh=2))

In [0]:
df

In [0]:
print(df.dropna(subset=[1, 2], how='any').shape)
print(df.dropna(subset=[1, 2], how='all').shape)

### Filling In Missing Data

In [0]:
df.fillna(0)

In [0]:
# Calling fillna with a dict, you can use a different fill value for each column

df.fillna({1: 0.5, 2: 0})

In [0]:
# fillna returns a new object, but you can modify the existing object in-place
_ = df.fillna(0, inplace=True)
df

In [0]:
df = pd.DataFrame(np.random.randn(6, 3))

In [0]:
df.iloc[2:, 1] = NA
df.iloc[4:, 2] = NA
df

In [0]:
df.fillna(method='ffill')

In [0]:
df.fillna(method='ffill', limit=2)

In [0]:
# we might pass mean or median of a series

data = pd.Series([1., NA, 3.5, NA, 7])

In [0]:
data.fillna(data.mean())

**Arguments** - **Description** for fillna

**value** - Scalar value or dict-like object to use to fill missing values

**method** - Interpolation; by default 'ffill' if function called with no other arguments

**axis** - Axis to fill on; default axis=0

**inplace** - Modify the calling object without producing a copy

**limit** - For forward and backward filling, maximum number of consecutive periods to fill

## Data Transformation

### Removing Duplicates

In [0]:
data = pd.DataFrame({'k1': ['one', 'two'] * 3 + ['two'],
                    'k2': [1, 1, 2, 3, 3, 4, 4]})
data

The DataFrame method **duplicated** returns a boolean Series indicating whether each row is a duplicate (has been observed in a previous row) or not.

In [0]:
data.duplicated()

In [0]:
# drop_duplicates returns a DataFrame where the duplicated array is False

data.drop_duplicates()

In [0]:
# Suppose we had an additional column of values and wanted to 
# filter duplicates only based on the 'k1' column

data['v1'] = range(7)
data

In [0]:
data.drop_duplicates(['k1'])

### Transforming Data Using a Function or Mapping

For many datasets, you may wish to perform some transformation based on the values in an array, Series, or column in a DataFrame.

In [0]:
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon',
                              'Pastrami', 'corned beef', 'Bacon',
                              'pastrami', 'honey ham', 'nova lox'],
                     'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

In [0]:
# Suppose you wanted to add a column indicating the 
# type of animal that each food came from

meat_to_animal = {
 'bacon': 'pig',
 'pulled pork': 'pig',
 'pastrami': 'cow',
 'corned beef': 'cow',
 'honey ham': 'pig',
 'nova lox': 'salmon'
}

In [0]:
lowercased = data['food'].str.lower()
lowercased

In [0]:
data['animal'] = lowercased.map(meat_to_animal)
# or
# data['food'].map(lambda x: meat_to_animal[x.lower()])
data

### Replacing Values

Filling in missing data with the fillna method is a special case of more general value replacement. As you’ve already seen, map can be used to modify a subset of values in an object but **replace** provides a simpler and more flexible way to do so

In [0]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])

In [0]:
data

In [0]:
print(data.replace(-999, np.nan))
print(data.replace([-999, -1000], np.nan))
print(data.replace([-999, -1000], [np.nan, 0]))
print(data.replace({-999: np.nan, -1000: 0}, inplace=True))

### Renaming Axis Indexes

axis labels can be transformed by a function or mapping of some form to produce new, differently labeled objects. 

In [0]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                    index=['Ohio', 'Colorado', 'New York'],
                    columns=['one', 'two', 'three', 'four'])

In [0]:
transform = lambda x: x[:4].upper()

# Like a Series, the axis indexes have a map method

print(data.index.map(transform))

# -----------
trans = lambda x: x[:2].upper()
print(data.index.map(trans))

In [0]:
data.index = data.index.map(transform)
data

If you want to create a transformed version of a dataset without modifying the original, a useful method is **rename**

In [0]:
data.rename(index=str.title, columns=str.upper)

In [0]:
# rename can be used in conjunction with a dict-like object
# providing new values for a subset of the axis labels

data.rename(index={'OHIO': 'INDIANA'},
            columns={'three': 'peekaboo'}, inplace=True)
data

In [0]:
# ToD: Discretization and Binning

### Detecting and Filtering Outliers

Filtering or transforming outliers is largely a matter of applying array operations.

In [0]:
data = pd.DataFrame(np.random.randn(1000, 4))
data.describe()

In [0]:
col = data[0]

In [0]:
col[np.abs(col) > 3]

To select all rows having a value exceeding 3 or –3, you can use the any method on a boolean DataFrame

In [0]:
data[(np.abs(data) > 3).any(axis=1)]

Values can be set based on these criteria. Here is code to cap values outside the interval –3 to 3

In [0]:
data[np.abs(data) > 3] = np.sign(data) * 3

In [0]:
data.describe()

In [0]:
# The statement np.sign(data) produces 1 and –1 values based on 
# whether the values in data are positive or negative

np.sign(data).head()

### Permutation and Random Sampling


Permuting (randomly reordering) a Series or the rows in a DataFrame is easy to do using the numpy.random.permutation function. Calling permutation with the length of the axis you want to permute produces an array of integers indicating the new ordering

In [0]:
df = pd.DataFrame(np.arange(20).reshape((5, 4)))

In [0]:
sampler = np.random.permutation(5)
sampler

That array can then be used in iloc-based indexing or the equivalent **take** function

In [0]:
df

In [0]:
df.take(sampler)

To select a random subset **without replacement**, you can use the **sample** method on Series and DataFrame

In [0]:
df.sample(n=3)

In [0]:
#  sample with replacement

choices = pd.Series([5, 7, -1, 6, 4])
draws = choices.sample(n=10, replace=True)
draws

### Computing Indicator/Dummy Variables


In [0]:
train = pd.read_csv('http://bit.ly/kaggletrain')
train.head()

In [0]:
# create a dummy column for Sex column

# using map method
train['Sex_male'] = train.Sex.map({'female': 0, 'male': 1})
train.head()

In [0]:
# using get_dummies
  
pd.get_dummies(train.Sex)

Generally, **k** categorical values, then use **k-1** dummy variables to represent it.

for Sex, we have to categorical values so we need only 1 variable to encode all the given information. So we drop the first column.

In [0]:
pd.get_dummies(train.Sex, prefix='Sex').iloc[:, 1:]

In [0]:
train.Embarked.value_counts()

In [0]:
embarked_dummies = pd.get_dummies(train.Embarked, prefix='Embarked').iloc[:, 1:]

In [0]:
train = pd.concat([train, embarked_dummies], axis=1)
train.head()

In [0]:
# pass columns to get_dummies makes it more easy

train = pd.read_csv('http://bit.ly/kaggletrain')

pd.get_dummies(train, columns=['Sex', 'Embarked'])

In [0]:
# drop the first column after get_dummies

pd.get_dummies(train, columns=['Sex', 'Embarked'], drop_first=True)

In [0]:
# example of MovieLens 1M dataset

mnames = ['movieid', 'title', 'genres']

In [0]:
movies = pd.read_table(r'MovieLens-1M/movies.dat', sep='::', header=None,
                       names=mnames, engine='python')

In [0]:
movies.head()

In [0]:
all_genres = []

for x in movies.genres:
    all_genres.extend(x.split('|'))

In [0]:
genres = pd.unique(all_genres)
genres

In [0]:
# One way to construct the indicator DataFrame is to start with a
# DataFrame of allzeros

zero_matrix = np.zeros((len(movies), len(genres)))

In [0]:
dummies = pd.DataFrame(zero_matrix, columns=genres)

Now, iterate through each movie and set entries in each row of dummies to 1. To do this, we use the dummies.columns to compute the column indices for each genre

In [0]:
gen = movies.genres[0]
print(gen.split('|'))
dummies.columns.get_indexer(gen.split('|'))

Then, we can use .iloc to set values based on these indices

In [0]:
for i, gen in enumerate(movies.genres):
    indices = dummies.columns.get_indexer(gen.split('|'))
    dummies.iloc[i, indices] = 1

In [0]:
# join with movies

movies_windic = movies.join(dummies.add_prefix('Genre_'))

In [0]:
movies_windic.iloc[0]

## String Manipulation
### Basic String methods

In [0]:
val = 'a,b, guido'

pieces = [x.strip() for x in val.split(',')]
pieces

In [0]:
"::".join(pieces)

**built-in String methods**

|Argument | Description    |
|---|---|
|count|Return the number of non-overlapping occurrences of substring in the string.     |
|endswith|Returns True if string ends with suffix.     |
|startswith|Returns True if string starts with prefix.    |
|join|Use string as delimiter for concatenating a sequence of other strings. |
|index|Return position of first character in substring if found in the string; raises ValueError if not found.  |
|find|Return position of first character of rst occurrence of substring in the string; like index, but returns –1  if          not found.|
|rfind|Return position of first character of last occurrence of substring in the string; returns –1 if not found.   |
|replace|Replace occurrences of string with another string.|
|strip, rstrip, lstrip|Trim whitespace, including newlines; equivalent to x.strip() (and rstrip, lstrip, respectively)
                      for each element|
|split|Break string into list of substrings using passed delimiter.    |
|lower|Convert alphabet characters to lowercase.    |
|upper|Convert alphabet characters to uppercase.   |
|casefold|Convert characters to lowercase, and convert any region-specific variable character combinations to a common comparable form.   
|ljust, rjust|Left justify or right justify, respectively; pad opposite side of string with spaces (or some other fill character) to return a string with a minimum width   |

### Regular Expressions

The re module functions fall into three categories: pattern matching, substitution,
and splitting. Naturally these are all related; a regex describes a pattern to locate in the
text, which can then be used for many purposes.

In [0]:
import re

In [0]:
text = "foo    bar\tbaz   \tqux"
text

In [0]:
re.split('\s+', text)

When you call re.split('\s+', text), the regular expression is first compiled, and then its split method is called on the passed text. You can compile the regex yourself with **re.compile**, forming a reusable regex object

Creating a regex object with re.compile is highly recommended if you intend to
apply the same expression to many strings; doing so will save CPU cycles

In [0]:
regex = re.compile('\s+')

In [0]:
regex.split(text)

In [0]:
text = """Dave dave@google.com
Steve steve@gmail.com
Rob rob@gmail.com
Ryan ryan@yahoo.com
"""
pattern = r'[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}'

# re.IGNORECASE makes the regex case-insensitive
regex = re.compile(pattern, flags=re.IGNORECASE)

In [0]:
regex.findall(text)

**match** and **search** are closely related to findall. While findall returns all matches
in a string, search returns only the first match. More rigidly, match only matches at
the beginning of the string

**search** returns a special match object for the first email address in the text. For the
preceding regex, the match object can only tell us the start and end position of the
pattern in the string

In [0]:
m = regex.search(text)

In [0]:
m

In [0]:
text[m.start(): m.end()]

In [0]:
print(regex.match(text))

In [0]:
print(regex.sub('REDACTED', text))

 segment each address into its three components: username, domain name, and domain suffix

In [0]:
pattern = r'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})'

In [0]:
regex = re.compile(pattern, flags=re.IGNORECASE)

In [0]:
m = regex.match('wesm@bright.net')
m.groups()

In [0]:
regex.findall(text)

In [0]:
print(regex.sub(r'Username: \1, Domain: \2, Suffix: \3', text))

**Regular expression methods**

|Argument| Description|
|-|-|
|findall| Return all non-overlapping matching patterns in a string as a list|
|finditer| Like findall, but returns an iterator|
|match |Match pattern at start of string and optionally segment pattern components into groups; if the pattern matches, returns a match object, and otherwise None|
|search| Scan string for match to pattern; returning a match object if so; unlike match, the match can be anywhere in the string as opposed to only at the beginning|
|split| Break string into pieces at each occurrence of pattern|
|sub, subn|Replace all (sub) or first n occurrences (subn) of pattern in string with replacement expression; use symbols \1, \2, ... to refer to match group elements in the replacement string|

### Vectorized String Functions in pandas

In [0]:
data = {'Dave': 'dave@google.com', 'Steve': 'steve@gmail.com',
        'Rob': 'rob@gmail.com', 'Wes': np.nan}

data = pd.Series(data)
data

In [0]:
data.isnull()

You can apply string and regular expression methods can be applied (passing a lambda or other function) to each value using data.map, but it will fail on the NA (null) values. To cope with this, Series has array-oriented methods for string operations that skip NA values. These are accessed through Series’s str attribute; for example,

In [0]:
data.str.contains('gmail')

Regular expressions can be used, too, along with any re options like IGNORECASE

In [0]:
pattern

In [0]:
data.str.findall(pattern, flags=re.IGNORECASE)

In [0]:
 matches = data.str.match(pattern, flags=re.IGNORECASE)

There are a couple of ways to do vectorized element retrieval. Either use **str.get** or index into the str attribute

In [0]:
matches